## Code to install MILC and QUDA on Summit
Oct, 2022

In [1]:
import os, glob
import subprocess as sp
import numpy as np
import shutil


In [2]:
from ipywidgets import *

## Build scripts

In [3]:
start_dir=os.getcwd()
print(start_dir)

# Create directory to save install scripts
if not os.path.exists(start_dir+'/install_scripts'):
    os.makedirs(start_dir+'/install_scripts/')

/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Staggered_multigrid_build/build_crusher


In [4]:
dict_pars={}
dict_pars['build_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/builds_crusher/install_dec1_2022'
dict_pars['run_dir']='/gpfs/alpine/lgt104/proj-shared/ayyar/runs_crusher/runs_dec1_2022'

## Create upper level directories
if not os.path.exists(dict_pars['build_dir']):
    os.makedirs(dict_pars['build_dir']+'/QUDA/src/')
    # os.makedirs(dict_pars['build_dir']+'/QUDA/src/quda/')
    # os.makedirs(dict_pars['build_dir']+'/MILC')
    os.makedirs(dict_pars['build_dir']+'/install')

In [5]:
### Setup env
env_bash_strg='''#!/bin/bash
#############

module load craype-accel-amd-gfx90a
module load cray-mpich/8.1.12
module load cmake
module load rocm/4.5.2
module list

export MPICH_ROOT=/opt/cray/pe/mpich/8.1.12
export MPICH_DIR=${{MPICH_ROOT}}/ofi/crayclang/10.0

## These must be set before running

#export SRCROOT=${{BUILD_DIR}}/../src
#export BUILDROOT=${{BUILD_DIR}}/build
export INSTALLROOT=${{BUILD_DIR}}/install
export GPU_TARGET=gfx90a

MPI_CFLAGS="-I${{MPICH_DIR}}/include -g"
MPI_LDFLAGS="-g -L${{MPICH_DIR}}/lib -lmpi -L${{MPICH_ROOT}}/gtl/lib -lmpi_gtl_hsa"

export PK_BUILD_TYPE="Release"

export PATH=${{ROCM_PATH}}/bin:${{PATH}}
export LD_LIBRARY_PATH=${{INSTALLROOT}}/quda/lib:${{MPICH_DIR}}/lib:${{MPICH_ROOT}}/gtl/lib:${{LD_LIBRARY_PATH}}

'''.format(**dict_pars)


In [6]:
build_bash_strg1='''#!/bin/bash
date
#############
## Source environment

script_loc=$BUILD_DIR/install_scripts
source $script_loc/setup_env_crusher.sh
WORKDIR=$BUILD_DIR

############
## Build QUDA
QUDA_SRC=${{WORKDIR}}/QUDA/src/quda
QUDA_BUILD=${{WORKDIR}}/QUDA/build

pushd .
if [ ! -d $QUDA_SRC ]
then
##  git clone --branch feature/hip-compile-fixes https://github.com/lattice/quda $QUDA_SRC # clone QUDA to the desired directory

git clone --branch develop https://github.com/lattice/quda $QUDA_SRC # clone QUDA to the desired directory

else
  cd $QUDA_SRC; git pull; cd ..
fi

if [ ! -d $QUDA_BUILD ]
then
  mkdir $QUDA_BUILD
else
  cd $QUDA_BUILD
  rm -rf *
fi

export QUDA_GPU_ARCH=gfx90a

cd $QUDA_BUILD
cmake -DCMAKE_BUILD_TYPE=DEVEL -DQUDA_DIRAC_DEFAULT_OFF=ON -DQUDA_DIRAC_STAGGERED=ON -DQUDA_GAUGE_ALG=ON -DQUDA_GAUGE_TOOLS=ON -DQUDA_DOWNLOAD_USQCD=ON -DQUDA_QIO=ON -DQUDA_QMP=ON -DQUDA_MULTIGRID=ON \
-DQUDA_TARGET_TYPE="HIP" \
-DROCM_PATH=${{ROCM_PATH}} \
-DGPU_TARGETS="gfx90a" \
-DCMAKE_CXX_COMPILER="hipcc" \
-DCMAKE_C_COMPILER="hipcc" \
-DCMAKE_CXX_FLAGS="${{MPI_CFLAGS}}" \
-DCMAKE_C_FLAGS="${{MPI_CFLAGS}}" \
-DCMAKE_EXE_LINKER_FLAGS="${{MPI_LDFLAGS}}" \
-DCMAKE_SHARED_LINKER_FLAGS="${{MPI_LDFLAGS}}" \
-DCMAKE_C_STANDARD=99 \
-DCMAKE_INSTALL_PREFIX=${{INSTALLROOT}}/quda \
$QUDA_SRC

nice make VERBOSE=1 -j 16 
make install
popd

date
'''.format(**dict_pars)


In [7]:
build_bash_strg2='''#!/bin/bash
#############
## Source environment
script_loc=$BUILD_DIR/install_scripts
source $script_loc/setup_env_crusher.sh

git clone --branch feature/staggMG https://github.com/weinbe2/milc_qcd ${{BUILD_DIR}}/milc_qcd
## git clone --branch develop https://github.com/milc-qcd/milc_qcd ${{BUILD_DIR}}/milc_qcd
'''.format(**dict_pars)

In [8]:
build_bash_strg3='''#!/bin/bash
date
#############
## Source environment
script_loc=$BUILD_DIR/install_scripts
source $script_loc/setup_env_crusher.sh

QUDA_INSTALL=${{INSTALLROOT}}/quda

LIBQUDA="-Wl,-rpath ${{QUDA_INSTALL}}/lib -L${{QUDA_INSTALL}}/lib -lquda -D__gfx90a --offload-arch=gfx90a --amdgpu-target=gfx90a -Wl,-rpath=${{ROCM_PATH}}/hiprand/lib -L${{ROCM_PATH}}/hiprand/lib -Wl,-rpath=${{ROCM_PATH}}/rocfft/lib -L${{ROCM_PATH}}/rocfft/lib -lhiprand -lrocfft -Wl,-rpath=${{ROCM_PATH}}/hipblas/lib -L${{ROCM_PATH}}/hipblas/lib -lhipblas -Wl,-rpath=${{ROCM_PATH}}/rocblas/lib -L${{ROCM_PATH}}/rocblas/lib -lrocblas -Wl,-rpath=${{ROCM_PATH}}/hip/lib"

echo $BUILD_DIR/milc_qcd/ks_spectrum
############ Make ks_spectrum_hisq ##################
cd $BUILD_DIR/milc_qcd/ks_spectrum
cp ../Makefile .
make clean

MY_CC=hipcc \
MY_CXX=hipcc \
ARCH="" \
COMPILER="gnu" \
OPT="-O3 --offload-arch=gfx90a" \
CUDA_HOME="" \
QUDA_HOME=${{QUDA_INSTALL}} \
WANTQUDA=true \
WANT_FN_CG_GPU=true \
WANT_FL_GPU=true \
WANT_GF_GPU=true \
WANT_FF_GPU=true \
WANT_MIXED_PRECISION_GPU=2 \
PRECISION=2 \
MPP=true \
OMP=true \
WANTQIO=true \
WANTQMP=true \
QIOPAR=${{INSTALLROOT}}/quda \
QMPPAR=${{INSTALLROOT}}/quda \
LIBQUDA=${{LIBQUDA}} \
OFFLOAD=HIP \
CGEOM="-DFIX_NODE_GEOM -DFIX_IONODE_GEOM" \
KSCGMULTI="-DKS_MULTICG=HYBRID -DMULTISOURCE -DMULTIGRID" \
CTIME="-DNERSC_TIME -DCGTIME -DFFTIME -DFLTIME -DGFTIME -DREMAP -DPRTIME -DIOTIME" \
make -j 1 ks_spectrum_hisq
cd ..

date
'''.format(**dict_pars)

In [9]:
loc1=start_dir+'/install_scripts/'
loc2=dict_pars['build_dir']+'/'
loc1,loc2

('/autofs/nccs-svm1_home1/venkitesh/projects/lqcd/multigrid_lattice/Staggered_multigrid_build/build_crusher/install_scripts/',
 '/gpfs/alpine/lgt104/proj-shared/ayyar/builds_crusher/install_dec1_2022/')

In [10]:
fname1='setup_env_crusher.sh'
with open (loc1+fname1,'w') as f: f.write(env_bash_strg)

fname2='build_quda.sh'
with open (loc1+fname2,'w') as f: f.write(build_bash_strg1)

fname3='build_milc1.sh'
with open (loc1+fname3,'w') as f: f.write(build_bash_strg2)

fname4='build_milc2.sh'
with open (loc1+fname4,'w') as f: f.write(build_bash_strg3)

print(fname1,fname2,fname3,fname4)

setup_env_crusher.sh build_quda.sh build_milc1.sh build_milc2.sh


In [ ]:
%%bash -s "$fname1" "$fname2" "$fname3" "$fname4" "$loc1" "$loc2" ## Use python variables in bash

# ls -al $5$1
# ls -al $5$4

chmod +x $5/$1
chmod +x $5/$2
chmod +x $5/$3
chmod +x $5/$4

mv $5 $6

### Run the build
cd \<build_dir\> \
export BUILD_DIR=$PWD \
./install_scripts/build_quda.sh 2>&1 | tee op_quda.out \
./install_scripts/build_milc1.sh 2>&1 | tee op_milc1.out \
< make some changes to files> \
./install_scripts/build_milc2.sh 2>&1 | tee op_milc2.out

In [ ]:
# cat install_scripts/build_milc.sh